<a href="https://colab.research.google.com/gist/oserikov/4406dea7f18328a5fe9e78d4e17bb9d5/gobot-md-yaml-configs-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Markdown & YAML to configure deeppavlov GO-bot

This tutorial notebook illustrates the way to configure a goal-oriented bot using 3 config files:
* `stories.md` (or `stories-{trn, tst, val}.md` but these are just subsamples)
* `nlu.md`
* `domain.yml`

The purpose of these files is briefly described below and the files formats desctiptions are below too.  
One could scroll down to Demo sections to take a look at the example files in the working scenarios.

## The purpose of `.md`, `.yml` config files

### `stories.md`



`stories.md` is to provide the **dialogues dataset** for model to train on. 

The dialogues are  
not the sequences of user utterances *texts* and respective system replies *texts*  
but the *intent + slots* user utterances *labels* and respective system replies *labels*.

This is so to distinguish the NLU-NLG tasks from the actual dialogues storytelling experience: one should be able to describe just the scripts of dialogues to the system.

#### format

Stories file is a markdown file of the following format:

```markdown
## story_title(not used by algorithm, but useful to work with for humans)
* user_action_label{"1st_slot_present_in_action": "slot1_value", .., "Nth_slot_present_in_action": "slotN_value"}
 - system_respective_utterance
* another_user_action_of_the_same_format
  - another_system_response
...

## another_story_title
...

```

**See examples below in this tutorial**

### `nlu.md`


`nlu.md` is contrariwise to provide the **NLU training set** irrespective of the dialogues scripts.

#### format

NLU file is a markdown file of the following format:

```markdown
## intent:possible_user_action_label_1
- An example of user text that has the possible_user_action_label_1 action label
- Another example of user text that has the possible_user_action_label_1 action label
...

## intent:possible_user_action_label_N
- An example of user text that has the (possible_user_action_label_N)[action_label] action label
<!-- Slotfilling dataset is provided as an inline markup of user texts -->
...

```

**See examples below in this tutorial**

### `domain.yml`

`domain.yml` is to desribe the task-specific domain and serves two purposes:
* provide the NLG templates
* provide some specific configuration of the NLU 

#### format

Domain file is a YAML file of the following format:

```yaml
# slots section lists the possible slot names (aka slot types) 
# that are used in the domain (i.e. relevant for bot's tasks)
# currently only type: text is supported
slots:
  slot1_name:
    type: text
  ...
  slotN_name:
    type: text

# entities list now follows the slots list 2nd level keys 
# and is present to support upcoming features. Stay tuned for updates with this!
entities:
- slot1_name
...
- slotN_name

# intents section lists the intents that can appear in the stories
# being kept together they do describe the user-side part of go-bot's experience
intents:
  - user_action_label
  - another_user_action_of_the_same_format
  ...

# responses section lists the system response templates.
# Despite system response' titles being usually informative themselves
#   (one could even find them more appropriate when no actual "Natural Language" is needed 
#    (e.g. for buttons actions in bot apps))
# It is though extremely useful to be able to serialize the response title to text. 
# That's what this section content is needed for.
responses:
  system_utterance_1:
    - text: "The text that system responds with"
  another_system_response:
    - text: "Here some text again"

```

**See examples below in this tutorial**

## Minimal demo

Let's build the simplest bot possible.  
Let in answer with greeting to greeting and with goodbye to goodbye.
And with "no worries" to thanks.

In [1]:
DP_MIN_DEMO_DIR = "dp_minimal_demo_dir"  # we will work in this folder

In [2]:
import os

%cd /content
os.makedirs(DP_MIN_DEMO_DIR, exist_ok=True)
%cd {DP_MIN_DEMO_DIR}

/content


`stories.md` here is pretty straightforward

In [4]:
%%writefile stories.md

## greet
* greet
  - utter_greet

## thank
* thank
  - utter_noworries

## goodbye
* bye
  - utter_bye

Writing stories.md


`nlu.md` just lists the possible ways to say things :)

In [5]:
%%writefile nlu.md

## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks

## intent:bye
- goodbye
- goodnight
- good bye
- good night
- see ya
- toodle-oo
- bye bye
- gotta go
- farewell

## intent:thank
- Thanks
- Thank you
- Thank you so much
- Thanks bot
- Thanks for that
- cheers


Writing nlu.md


`domain.yml` lists 
* the possible user action intents 
* the possible system response actions

Entities and slots are omitted for the moment. See the more sophisticated example below for intuition on them.

In [6]:
%%writefile domain.yml

intents:
  - greet
  - bye
  - thank

responses:
  utter_noworries:
  - text: No worries!
  utter_greet:
  - text: Hi
  utter_bye:
  - text: Bye!

Writing domain.yml


Let's install the `deeppavlov` library.

In [7]:
!pip install git+https://github.com/deepmipt/DeepPavlov.git@gobot-md-yaml-config-fixes
!python -m deeppavlov install gobot_simple_dstc2

  Cloning https://github.com/deepmipt/DeepPavlov.git (to revision gobot-md-yaml-config-fixes) to /tmp/pip-req-build-es08lk40
  Running command git clone -q https://github.com/deepmipt/DeepPavlov.git /tmp/pip-req-build-es08lk40
  Running command git checkout -b gobot-md-yaml-config-fixes --track origin/gobot-md-yaml-config-fixes
  Switched to a new branch 'gobot-md-yaml-config-fixes'
  Branch 'gobot-md-yaml-config-fixes' set up to track remote branch 'gobot-md-yaml-config-fixes' from 'origin'.
     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 2.1MB 18.3MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 1.5MB 42.3MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 10.4MB 41.2MB/s 
     |████████████████████████████████| 512kB 39.8MB/s 
     |████████████████████████████████| 7.3MB 19.2MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
   

2020-07-29 10:44:41.851 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'gobot_simple_dstc2' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/go_bot/gobot_simple_dstc2.json'
     |████████████████████████████████| 110.5MB 98kB/s 
     |████████████████████████████████| 512kB 39.9MB/s 
     |████████████████████████████████| 3.8MB 38.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8c41178fbd0d8ad07b1932ecd1a9f3a607949597d937b5eb86700882281ac439
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling te

Let's specify the path to our dataset (the folder we are in right now) and the folder where to store the trained bot.

In [8]:

from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)

gobot_config['metadata']['variables']['DATA_PATH'] = '.'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

!echo "{}" > dstc2-actions2slots.json  # a hack that will be redundant in future versions. keep an empty json file in the data folder

Since our data is basically the mock tutorial data we will use the same subsamples for all of train, test and valid subsamples. 

Our goal here is not the brilliant statistical measures but the working *toy* system. More sophisticated examples will be available soon.

In [9]:
!cp stories.md stories-trn.md
!cp stories.md stories-tst.md 
!cp stories.md stories-val.md 

Let's train the bot

In [11]:
from deeppavlov import train_model

train_model(gobot_config, download=True)

2020-07-29 10:46:15.31 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt
347MB [00:16, 20.8MB/s]
2020-07-29 10:46:33.314 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/datasets/gobot_md_yaml_minimal.tar.gz to /content/gobot_md_yaml_minimal.tar.gz
100%|██████████| 472/472 [00:00<00:00, 818kB/s]
2020-07-29 10:46:33.417 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /content/gobot_md_yaml_minimal.tar.gz archive into /content/dp_minimal_demo_dir
2020-07-29 10:46:33.447 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 10:46:33.453 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_

2020-07-29 10:46:37.520 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 10:46:37.527 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpgquo4cpm]
2020-07-29 10:46:37.530 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp9ecfovf6]
2020-07-29 10:46:37.533 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpa1uvr_mo]
2020-07-29 10:46:37.654 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]






Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-07-29 10:47:24.675 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-07-29 10:47:24.749 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best per_item_action_accuracy of 0.5


{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 0.5}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


...

2020-07-29 10:48:11.348 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-07-29 10:48:11.351 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 10:48:11.359 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp46mip4xp]
2020-07-29 10:48:11.363 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpj8fgl_gg]


{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}


2020-07-29 10:48:11.367 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp4iw165sq]
2020-07-29 10:48:11.369 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 10:48:55.514 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-07-29 10:48:55.519 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_minimal_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


Chainer[<deeppavlov.models.go_bot.wrapper.DialogComponentWrapper at 0x7ff74f93ca20>,

Let's build our bot to play with it

In [12]:
from deeppavlov import build_model
bot = build_model(gobot_config)

2020-07-29 10:48:55.595 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-07-29 10:48:55.598 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_minimal_demo_dir/domain.yml
2020-07-29 10:48:55.605 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpr4_7z4ts]
2020-07-29 10:48:55.608 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmphwq8nzhx]
2020-07-29 10:48:55.610 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpy87h0mkq]
2020-07-29 10:48:55.613 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlo

INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


In [13]:
bot.reset()

bot(["start"])
bot(["Hi"])[0][0].actions_tuple

('utter_greet',)

The bot answers with greeting to greeting. What will it respond to some grateful message?

In [14]:
bot.reset()
bot(["start"])

bot(["Thanks!"])[0][0].actions_tuple

('utter_noworries',)

Ok, no woories is an expected response. And with goodbye message, let's see some textual answer.

In [15]:
bot.reset()
bot(["start"])

bot_response_actions = bot(["bye"])[0][0].actions_tuple

In [16]:
import yaml

system_utter2text = yaml.load(open("domain.yml"))["responses"]
system_utter2text[bot_response_actions[0]][0]["text"]

'Bye!'

## Not that minimal demo

Now let's take the [Schema](asdf) dataset.
We naively converted most of *Restaurants 1* dialogues from Schema to the md+yml format to keep things simple. 

These are the annotated dialogs on the restaurant tables booking domain.
We will not involve the DB usage in our example and the bot should just learn to respond with relevant actions to user's utterances.  

In [17]:
DP_BIG_DEMO_DIR = "dp_big_demo_dir"  # we'll work in this directory

In [18]:
import os

%cd /content
os.makedirs(DP_BIG_DEMO_DIR, exist_ok=True)
%cd {DP_BIG_DEMO_DIR}

/content
/content/dp_big_demo_dir


download the data

In [19]:
# let's get the mentioned converted Schema-dataset subset
!wget http://files.deeppavlov.ai/datasets/schema_resto_md_yaml.tar.gz
!tar -zxf schema_resto_md_yaml.tar.gz 

--2020-07-29 10:49:41--  http://files.deeppavlov.ai/datasets/schema_resto_md_yaml.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98718 (96K) [application/octet-stream]
Saving to: ‘schema_resto_md_yaml.tar.gz’

schema_resto_md_yam 100%[===================>]  96.40K  --.-KB/s    in 0.09s   

2020-07-29 10:49:42 (1.10 MB/s) - ‘schema_resto_md_yaml.tar.gz’ saved [98718/98718]



First of all let's train the NER model that will be used by bot. No .md or .yml configs illustrated here, but the trained model is inevitable part of the go-bot.

In [20]:
from deeppavlov import configs, train_model
from deeppavlov.core.common.file import read_json

ner_config = read_json(configs.ner.ner_conll2003)
ner_config["dataset_reader"] = read_json(configs.ner.ner_few_shot_ru)["dataset_reader"]
ner_config["dataset_reader"]["data_path"] = "schema_resto_md_yaml/ner"

In [21]:
!python -m deeppavlov download ner_conll2003
!rm -r /root/.deeppavlov/models/ner_conll2003  # remove cached models if present

2020-07-29 10:49:46.783 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_conll2003' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_conll2003.json'
2020-07-29 10:49:47.873 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt?config=ner_conll2003 download because of matching hashes
2020-07-29 10:49:47.963 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_conll2003_v5.tar.gz?config=ner_conll2003 to /root/.deeppavlov/ner_conll2003_v5.tar.gz
100% 3.12M/3.12M [00:00<00:00, 3.81MB/s]
2020-07-29 10:49:48.874 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/ner_conll2003_v5.tar.gz archive into /root/.deeppavlov/models


In [22]:
# if rm says that these files are missing that's ok 
#   (we just deleted the whole folder containing these paths ^__^)
# but if they're present we should delete them 
# cause they're checkpoints of some other model
!rm /root/.deeppavlov/models/ner_conll2003/{checkpoint,model_no_pos}*

rm: cannot remove '/root/.deeppavlov/models/ner_conll2003/checkpoint*': No such file or directory
rm: cannot remove '/root/.deeppavlov/models/ner_conll2003/model_no_pos*': No such file or directory


In [ ]:
ner_model = train_model(ner_config)

2020-07-29 10:49:53.173 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 10:49:53.182 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 10:49:53.204 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-07-29 10:49:53.208 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]



Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use `tf.cast` instead.



2020-07-29 10:50:36.584 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2020-07-29 10:50:36.699 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-07-29 10:50:39.516 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 397: processed 1424 tokens with 32 phrases; found: 1152 phrases; correct: 0.

precision:  0.61%; recall:  21.88%; FB1:  1.18

	city: precision:  0.00%; recall:  0.00%; F1:  0.00 3

	cuisine: precision:  2.29%; recall:  46.15%; F1:  4.36 262

	date: precision:  0.00%; recall:  0.00%; F1:  0.00 1

	price_range: precision:  0.00%; recall:  0.00%; F1:  0.00 271

	restaurant_name: precision:  0.00%; recall:  0.00%; F1:  0.00 382

	time: precision:  0.43%; recall:  33.33%; F1:  0.85 233


2020-07-29 10:50:39.521 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best ner_f1 of 1.1824


...

{"valid": {"eval_examples_count": 214, "metrics": {"ner_f1": 89.2308, "ner_token_f1": 90.1961}, "time_spent": "0:00:01"}}


2020-07-29 10:53:13.660 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 397: processed 1846 tokens with 50 phrases; found: 53 phrases; correct: 0.

precision:  77.36%; recall:  82.00%; FB1:  79.61

	city: precision:  84.62%; recall:  84.62%; F1:  84.62 13

	cuisine: precision:  92.86%; recall:  92.86%; F1:  92.86 14

	date: precision:  50.00%; recall:  50.00%; F1:  50.00 6

	price_range: precision:  100.00%; recall:  100.00%; F1:  100.00 3

	restaurant_name: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	time: precision:  64.71%; recall:  84.62%; F1:  73.33 17




{"test": {"eval_examples_count": 267, "metrics": {"ner_f1": 79.6117, "ner_token_f1": 83.0189}, "time_spent": "0:00:01"}}


2020-07-29 10:53:13.986 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 10:53:13.989 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 10:53:13.992 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-07-29 10:53:13.994 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 10:53:57.616 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTM

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


Ner performs OK for everything except the restaurants' name identification.  
That's mostly because we avoid the usage of restaurants database here.

Let's check the NER performance running some slotfilling experiment

In [24]:
import json

json.dump(ner_config, open('ner_config.json', 'wt'))

In [25]:
import json
from deeppavlov import configs

from deeppavlov.core.common.file import read_json
from deeppavlov import build_model

slotfill_config = read_json(configs.ner.slotfill_dstc2)
slotfill_config['dataset_reader']['class_name'] = "md_yaml_dialogs_reader"
slotfill_config['metadata']['variables']['DATA_PATH'] = 'schema_resto_md_yaml'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'schema_resto_md_yaml/slotfill.json'
slotfill_config["chainer"]["pipe"][-1]["load_path"] = "schema_resto_md_yaml/slotfill.json"
slotfill_config["metadata"]["variables"]["NER_CONFIG_PATH"] = "ner_config.json"
slotfiller = build_model(slotfill_config, download=False)


2020-07-29 10:54:00.86 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 10:54:00.90 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 10:54:00.93 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-07-29 10:54:00.95 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 10:54:43.643 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 10:54:45.925 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.


In [26]:
slotfiller(["i'm looking for a thai food somewhere in SFO"])

[{'city': 'SFO', 'cuisine': 'Thai'}]

Seems OK. Let's save our slotfiller config and train and evaluate the restaurants bot, finally.

In [27]:
import json

json.dump(slotfill_config, open('slotfill_config.json', 'wt'))

In [28]:

from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)
gobot_config['chainer']['pipe'][-1]['slot_filler'] = {"config_path": "slotfill_config.json"}

gobot_config['metadata']['variables']['DATA_PATH'] = 'schema_resto_md_yaml'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

In [29]:
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-trn.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-tst.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-val.md

In [30]:
!echo "{}" > schema_resto_md_yaml/dstc2-actions2slots.json  # a hack that will be redundant in future versions. keep an empty json file in the data folder

In [31]:
from deeppavlov import train_model

train_model(gobot_config, download=True);

2020-07-29 10:54:54.943 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz to /root/.deeppavlov/slotfill_dstc2.tar.gz
100%|██████████| 641k/641k [00:00<00:00, 2.46MB/s]
2020-07-29 10:54:55.323 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/slotfill_dstc2.tar.gz archive into /root/.deeppavlov/models
2020-07-29 10:54:56.226 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt download because of matching hashes
2020-07-29 10:54:56.368 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/datasets/gobot_md_yaml_minimal.tar.gz to /content/dp_big_demo_dir/gobot_md_yaml_minimal.tar.gz
100%|██████████| 472/472 [00:00<00:00, 949kB/s]
2020-07-29 10:54:56.475 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /content/dp_big_demo_dir/gob

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 10:55:45.678 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-07-29 10:55:45.681 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 10:56:29.802 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-07-29 10:56:33.863 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best per_item_action_accuracy of 0.006


{"valid": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.006}, "time_spent": "0:00:05", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.5682}, "time_spent": "0:00:20", "epochs_done": 3, "batches_seen": 40, "train_examples_seen": 311, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.9333688497543335}}


2020-07-29 10:56:52.976 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best per_item_action_accuracy of 0.5682
2020-07-29 10:56:52.977 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
2020-07-29 10:56:52.978 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/dp_big_demo_dir/model/policy]


...

{"test": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.7033}, "time_spent": "0:00:04"}}


2020-07-29 11:00:08.755 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpakiascde]
2020-07-29 11:00:08.916 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmps4_56t5l]
2020-07-29 11:00:09.70 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmplfhk6fhg]
2020-07-29 11:00:09.315 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-07-29 11:00:09.347 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-07-29 11:00:09.349 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-07-29 11:00:09.353 INFO in 'deeppavlov.models.embedd

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 11:00:55.493 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-07-29 11:00:55.496 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 11:01:39.814 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-07-29 11:01:39.821 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [32]:
bot = build_model(gobot_config)

2020-07-29 11:01:40.15 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_big_demo_dir/word.dict]
2020-07-29 11:01:40.21 INFO in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 123: INSIDE MLU_MD_DialogsDatasetReader._read_domain_knowledge(): reading domain knowledge from /content/dp_big_demo_dir/schema_resto_md_yaml/domain.yml
2020-07-29 11:01:40.241 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpmwa809ja]
2020-07-29 11:01:40.383 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmppq1i2uhw]
2020-07-29 11:01:40.523 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmp34i0scb1]
2020-07-29 11:01:40.774 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/mode

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-07-29 11:02:26.582 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-07-29 11:02:26.585 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-07-29 11:03:10.462 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-07-29 11:03:10.468 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [33]:
bot.reset()

bot(["Hey!"])[0][0].actions_tuple

('utter_hi',)

Ok the bot performs OK on greeting

In [34]:
bot(["I'd like to find a restaurant for this evening"])[0][0].actions_tuple

('utter_REQUEST_City',)

The bot asks for an inevitably necessary information.

In [35]:
bot(["Somewhere in Oakland and for two people please"])[0][0].actions_tuple

('utter_OFFER_RestaurantName', 'OFFER_City')

The suggests some restaurant but since no actual DB usage happened that's just the fact that the bot knows that some slots are enough to book a table.

In [36]:
bot(["Cool! That's what I was looking for, thanks!"])[0][0].actions_tuple

('utter_INFORM_HasLiveMusic',)

Let's say goodbye to our bot

In [37]:
bot(["Bye bot"])[0][0].actions_tuple

('utter_REQUEST_Time',)

After the goodbye bot does perform uninterpretable. That's what most of recurrent system do after the end of the sequence cause no one reaches that poing. 

In [38]:
bot(["Bye bot"])[0][0].actions_tuple

('utter_CONFIRM_RestaurantName',
 'CONFIRM_City',
 'CONFIRM_Time',
 'CONFIRM_Date',
 'CONFIRM_PartySize')

## Final words

More of in-depth examples of goal-oriented bots and their features are coming with future releases, stay tuned!